In [ ]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import torch
import torchaudio

from synthmap.synth import Snare808

%load_ext autoreload
%autoreload 2

In [ ]:
snare = Snare808(48000, 48000)

num_params = snare.get_num_params()
params = torch.rand(8, num_params)

y = snare(params)

ipd.display(ipd.Audio(y, rate=48000))

In [ ]:
mel = torchaudio.transforms.MelSpectrogram(
    sample_rate=48000, n_fft=2048, hop_length=512, n_mels=128
)
x_mel = mel(y)
print(x_mel.shape)

In [ ]:
def log_distance(x, y):
    x = torch.log(x + 1.0)
    y = torch.log(y + 1.0)
    return torch.mean((torch.abs(x - y)))

In [ ]:
dist = torch.zeros(8, 8)
for i in range(8):
    for j in range(8):
        dist[i, j] = log_distance(x_mel[i], x_mel[j])

plt.imshow(dist)

In [ ]:
z = torch.randn(8, 16)

In [ ]:
latent_dist = torch.cdist(z, z, p=2)
plt.imshow(latent_dist)

In [ ]:
latent_dist.shape

In [ ]:
reg_loss = torch.tanh(latent_dist) - dist
reg_loss = torch.mean(torch.abs(reg_loss))
print(reg_loss)